In [2]:
from ultralytics import YOLO
# pip install ensemble-boxes
from ensemble_boxes import weighted_boxes_fusion

In [3]:
pt_model1 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch62.pt")  # load a pretrained model (recommended for training)
pt_model2 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch65.pt")  # load a pretrained model (recommended for training)

In [4]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('../../../data/images/image_5095.jpg')
im2 = Image.open('../../../data/images/image_5100.jpg')

In [5]:
result1 = pt_model1.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result1_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result1]
result1_n = [tuple(zip(*r)) for r in result1_n]
result1 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result1]
result1 = [tuple(zip(*r)) for r in result1]  # list of tuple[box, conf] in each image in xyxy format

C:\Program Files\Python39\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0: 928x1600 7 targets, 137.5ms
1: 928x1600 7 targets, 137.5ms
Speed: 8.5ms preprocess, 137.5ms inference, 809.1ms postprocess per image at shape (1, 3, 928, 1600)


In [6]:
result2 = pt_model2.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result2_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result2]
result2_n = [tuple(zip(*r)) for r in result2_n]
result2 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result2]
result2 = [tuple(zip(*r)) for r in result2]  # list of tuple[box, conf] in each image in xyxy format


0: 928x1600 7 targets, 60.0ms
1: 928x1600 7 targets, 60.0ms
Speed: 10.0ms preprocess, 60.0ms inference, 1.5ms postprocess per image at shape (1, 3, 928, 1600)


In [30]:
boxes_list = [[r[0] for r in result1_n[0]], [r[0] for r in result2_n[0]]]
scores_list = [[r[1] for r in result1_n[0]], [r[1] for r in result2_n[0]]]
labels_list = [[0] * len(result1_n[0]), [0] * len(result2_n[0])]
weights = [1, 1]
boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=0.5, skip_box_thr=1e-4)
boxes = boxes.tolist()
# normalize
boxes = [[x1*1520, y1*870, x2*1520, y2*870] for x1, y1, x2, y2 in boxes]
scores = scores.tolist()
wbf_result = [list(zip(boxes, scores))]

In [31]:
# plot
for im, boxes in zip([im1, im2], wbf_result):
    im = im.copy()
    draw = ImageDraw.Draw(im)
    for (x1, y1, x2, y2), conf in boxes:
        draw.rectangle(xy=((x1, y1), (x2, y2)), outline='red')
        draw.text((x1, y1), text=f'{conf:.2f}', fill='red')
    im.show()